In [24]:
import numpy as np

width, height, depth = 100, 100, 100

def create_box(width, height, depth):
    box = np.zeros((width, height, depth), dtype=bool)
    return box

def aggregate_unfolded_proteins_in_box(box):
    updated_box = box.copy()
    width, height, depth = box.shape
    
    #TODO: instead of for loop consider a 3x3 kernel convolution that calculates the sum. If the sum is greater than 1 aggregate?
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            for z in range(1, depth - 1):
                if box[x, y, z] and box[x-1, y, z] and box[x+1, y, z]:
                    updated_box[x, y, z] = -1
                elif box[x, y, z] and box[x, y-1, z] and box[x, y+1, z]:
                    updated_box[x, y, z] = -1
                elif box[x, y, z] and box[x, y, z-1] and box[x, y, z+1]:
                    updated_box[x, y, z] = -1
    
    return updated_box

def unfold_proteins_in_box(box, percentage=0.1):

    width, height, depth = box.shape

    standard_deviation = 1
    box2 = np.random.normal(loc=0, scale=standard_deviation, size=(width, height, depth))

    #wherever box2 is greater than 2.5 consider as unfolded - use Z statstitics instead later on to go from % to cutoff
    indices = np.where(box2 > 2.5)
    box[indices] = True
    return box

# Example usage
box = create_box(width, height, depth)
steps = 8

for i in range(steps):
    box = unfold_proteins_in_box(box)
    box = aggregate_unfolded_proteins_in_box(box)
    print(f"Step {i+1}: {np.sum(box)} proteins aggregated")
    
#TODO:
    #1. Instead of filling the grid with proteins, create a realistic size box based on volume 
    #2. Chop the box into voxels, size of a globular protein
    #3. Then use the concentration and average size of globular protein to fill a percentage of the box
    #4. Then use the above code to unfold and aggregate the proteins
    #5. Consider the size of an unfolded protein, it might span more then one voxel - thereby increasing chances of aggregation
    #7. improve speed by randomly drawing indices of the box instead of drawing all random numbers
    #8. Aggregation could be modeled not by direct neighbors, but two aggregated proteins in a 5x5 box.
        #8a. run a 5x5x5 kernel convolution that calculates the sum. If the sum is greater than 1 aggregate?
    #9. Randomly sample a % of Indices of all folded proteins to re-fold

#Needed objects:
    #1. General volume
    #2. state of voxel: empty, contains folded, unfolded, or aggregated
    #3. unfolding probability
    #4. aggregation probability
    #5. re-folding probability
    #6. size of a globular protein
    #7. concentration of proteins
    



Step 1: 6292 proteins aggregated
Step 2: 12509 proteins aggregated
Step 3: 18715 proteins aggregated
Step 4: 24750 proteins aggregated
Step 5: 30764 proteins aggregated
Step 6: 36863 proteins aggregated
Step 7: 42899 proteins aggregated
Step 8: 48906 proteins aggregated


In [9]:
import numpy as np
#Make the box:
#1 ml is 1 cm^3
box_size = 0.01 #m

protein_concentration = 10**-6 #1 uM

avogadro = 6.02214076e23 #1/mol

number_of_proteins = protein_concentration * box_size**3 * avogadro

#or from weight
#each protein weighs 80kda or 1.32843e-19 g
#3.5mg/ml as example
number_of_proteins = 3.5 / (1.32843e-19 * 1000)

#Diameter of protein is roughly 90 Angstrom
protein_volume = 4/3 * np.pi * (90e-10)**3 #m^3

#Number of voxels in the box
number_of_voxels = box_size**3 / protein_size
print(number_of_voxels)

print("% of filled voxels: ", number_of_proteins / number_of_voxels * 100)



1.1111111111111113e+22
% of filled voxels:  0.00023712201621462923


0